# TensorBoard

TensorBoard provides a suite of visualization tools to make it easier to understand, debug, and optimize Edward programs. You can use it "to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it"
([tensorflow.org](https://www.tensorflow.org/get_started/summaries_and_tensorboard)).

To use TensorBoard, we need to specify a directory for storing logs during inference. For example, if manually controlling inference, call
```python
inference.initialize(logdir='log')
```
or if using the catch-all `inference.run()`, include `logdir` as an argument.
As inference runs, files are outputted to `log/` within the working directory.
In commandline, we run TensorBoard and point to that directory.
```bash
tensorboard --logdir=log/
```
The command will provide a web address to access TensorBoard.
By default, it is http://localhost:6006.

You're set up! To prettify TensorBoard, we need to ensure we have 
proper naming of the log directory and tensors in the computational graph.

### Directory Naming

The following variables are set when either calling `inference.initialize` or `inference.run`:

+ `logdir`: The directory where all logs for inference are stored. For running inference and comparing across many hyperparameter configurations, we recommend a path such as `logdir='log/' + hyperparam_str`, where `hyperparam_str` is a string such as `batch_size_256_n_samples_5`.
+ `log_timestamp` (bool): If True (which is by default), a UTC time-stamped subdirectory is placed in `logdir` in the format 'YYYYMMDD_HHMMSS". This guarantees each run is stored in a unique subdirectory name per run so that TensorBoard can properly organize each run's information.
       
### Tensor Naming

Each TensorFlow tensor has a unique name, and TensorBoard presents information under these names.

+ __Name Scopes and Variable Scopes__: Use scoping to group tensors by adding a prefix to their names.
+ __Name Method__: Use the argument `name="new_name"` when defining a Edward random variable or TensorFlow tensor.

### TensorBoard

Run TensorBoard:
```bash
tensorboard --logdir=log/
```

Then go to the linked address. By default, it is http://localhost:6006. 
Here are some screenshots of what you should see.

**TensorBoard Scalars**
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard_scalars.png)

While the model is running you will see the variables values update in the graphs in the TensorBoard Scalars Tab.  You will see timeseries of the scalar variables in your model as well as other inference parameters relating to the specific loss functions and gradients used.

**TensorBoard Distributions**
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard_distributions.png)

The Distributions Tab will show how distributions of all of the values for a given variable over time.

**TensorBoard Histograms**
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard_histograms.png)

The Histograms is like the Distributions Tab except it shows the distributions over time as a three-dimensional chart.

**TensorBoard Graphs**
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard_graphs.png)

By cleaning up the variable naming as above you get a really clean image of the models in tensorflow.  Here you can see the default view that groups variables by their ``name_scope``.

**TensorBoard Graphs (Close-Up)**
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard_graph1.png)

By clicking into the nodes in the graph, you can see the detailed model structure. The more you name variables using custom names and use ``/`` to create a hierarchy in your naming convention, the cleaner this graphic will look.

## Example: Supervised Regression

In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from edward.models import Normal

plt.style.use('ggplot')

## Data

Simulate training and test sets of $40$ data points. They comprise of
pairs of inputs $\mathbf{x}_n\in\mathbb{R}^{5}$ and outputs
$y_n\in\mathbb{R}$. They have a linear dependence with normally
distributed noise.

In [2]:
def build_toy_dataset(N, w):
  D = len(w)
  x = np.random.normal(0.0, 2.0, size=(N, D))
  y = np.dot(x, w) + np.random.normal(0.0, 0.01, size=N)
  return x, y

ed.set_seed(42)

N = 40  # number of data points
D = 5  # number of features

# Variable scope adds this prefix to all data regardless if keyword 'name' is set
with tf.variable_scope('data'): 
  w_true = np.random.randn(D) * 0.5
  X_train, y_train = build_toy_dataset(N, w_true)
  X_test, y_test = build_toy_dataset(N, w_true)

## Model

Posit the model as Bayesian linear regression (Murphy, 2012).
For a set of $N$ data points $(\mathbf{X},\mathbf{y})=\{(\mathbf{x}_n, y_n)\}$,
the model posits the following distributions:

\begin{align*}
  p(\mathbf{w})
  &=
  \text{Normal}(\mathbf{w} \mid \mathbf{0}, \sigma_w^2\mathbf{I}),
  \\[1.5ex]
  p(b)
  &=
  \text{Normal}(b \mid 0, \sigma_b^2),
  \\
  p(\mathbf{y} \mid \mathbf{w}, b, \mathbf{X})
  &=
  \prod_{n=1}^N
  \text{Normal}(y_n \mid \mathbf{x}_n^\top\mathbf{w} + b, \sigma_y^2).
\end{align*}

The latent variables are the linear model's weights $\mathbf{w}$ and
intercept $b$, also known as the bias.
Assume $\sigma_w^2,\sigma_b^2$ are known prior variances and $\sigma_y^2$ is a
known likelihood variance. The mean of the likelihood is given by a
linear transformation of the inputs $\mathbf{x}_n$.

Let's build the model in Edward, fixing $\sigma_w,\sigma_b,\sigma_y=1$.

In [3]:
# Name scope only adds this prefix to names where keyword 'name' is set
with tf.name_scope('model'): 
  X = tf.placeholder(tf.float32, [N, D], name="X")
  w = Normal(loc=tf.zeros(D, name="weights/loc"), scale=tf.ones(D, name="weights/loc"), name="weights")
  b = Normal(loc=tf.zeros(1, name="bias/loc"), scale=tf.ones(1, name="bias/scale"), name="bias")
  y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N, name="y/scale"), name="y")

Here, we define a placeholder `X`. During inference, we pass in
the value for this placeholder according to data.

## Inference

We now turn to inferring the posterior using variational inference.
Define the variational model to be a fully factorized normal across
the weights.

In [1]:
with tf.name_scope('posterior'):
  qw = Normal(loc=tf.Variable(tf.random_normal([D]), name="qw/loc"),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([D])), name="qw/scale"), name="qw")
  qb = Normal(loc=tf.Variable(tf.random_normal([1]), name="qb/loc"),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([1])), name="qb/scale"), name="qb")

# Optionally create an 'inference' name_scope.  
# If it is absent, the charts are grouped nicely by 'parameters', 'gradient_norm' and 'loss'
# If it is added, TensorBoard Graph is slightly more organized.
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})   
inference.run(n_samples=5, n_iter=250, logdir='log')

NameError: name 'D' is not defined

## Acknowledgments

We thank Sean Kruzel for writing the initial version of this
tutorial.

A TensorFlow tutorial to TensorBoard can be found 
[here](https://www.tensorflow.org/get_started/summaries_and_tensorboard).